# MTurk: Approve / Reject Assignments + Post-Task Completion Processing of HITs

In [1]:
import boto3
import json
import pandas as pd
import ast
from simple_colors import *
from typing import Dict
import re
import pprint
import string
from pathlib import Path
import os

In [2]:
from mturk_processing_utils import (
    programmatically_review_HITs,
    get_dataset, 
    get_hit_id_dict, 
    parse_batch, 
    approved_parsed_batch_2_dicts,
    view_assignment,
    mturk,
    get_completetion_progress
)

In [3]:
PROJECT_ROOT_DIR = '/Users/nehasrikanth/Documents/paraphrase-nlu/'
dataset_name = ['atomic', 'snli', 'social']


raw_data_paths = {d: os.path.join(PROJECT_ROOT_DIR, f'raw-data/defeasible-nli/defeasible-{d}') for d in dataset_name}

### Social

In [ ]:
view_assignment('3BQU611VFPKO19HOP23FTHE4SIE990', dnli_social, hit_id_2_example_id_social_4)


In [ ]:
response = mturk.approve_assignment(
      AssignmentId='3BQU611VFPKO19HOP23FTHE4SIE990',
      RequesterFeedback='Thanks for doing a great job!',
)

In [ ]:
response = mturk.reject_assignment(
    AssignmentId='3H7XDTSHKCRESFH4WJJHZI82UV3WGB',
    RequesterFeedback='These are not all grammatical.'
)

In [ ]:
programmatically_review_HITs(
    get_hit_id_dict(
        'mturk_data/creation/social_dnli_annotation_examples_5.json',
    )[1], 
    get_dataset(raw_data_paths['social'], 'social')
)


In [ ]:
response = mturk.create_worker_block(
    WorkerId='A2D1P8V5YPI6G6',
    Reason='Worker is spamming or did not complete the task according to instructions.'
)

## Post Processing